In [26]:
import pandas
import numpy
import yaml
import xarray
import geopandas as gpd

In [16]:
case='whidbey'
model_var = 'DOXG'
MinDO_full={}
MinDO={}
with open(f'../etc/SSM_config_{case}.yaml', 'r') as file:
    ssm = yaml.safe_load(file)
    shp = ssm['paths']['shapefile']
gdf = gpd.read_file(shp)
gdf = gdf.rename(columns={'region_inf':'Regions'})
regions = gdf[['node_id','Regions']].groupby(
    'Regions').count().index.to_list()
regions.remove('Other')

In [36]:
gdf[['Regions','volume']].groupby('Regions').count()

,volume
Regions,
Hood,401
Main,893
Other,29
SJF_Admiralty,774
SOG_Bellingham,488
South_Sound,1028
Whidbey,531


In [18]:
# Load minimum DO results from reference case
reference = ssm['run_information']['reference']
base_dir = '/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/'
reference_file = f'{base_dir}/data/{case}/DOXG/{reference}/wc/daily_min_DOXG_wc.nc'
with xarray.open_dataset(reference_file) as ds:
    MinDO_full[reference]=ds[f'{model_var}_daily_min_wc']
    # Sub-sample nodes (from 16012 nodes to 7494)
    MinDO[reference]=MinDO_full[reference][:,:,gdf['node_id']-1]

# Get number of days and nodes
[ndays,nlevels,nnodes]=MinDO[reference].shape
ndays

361

In [19]:
dti = pandas.date_range("2014-01-01", periods=366, freq="D")

In [20]:
dti.shape

(366,)

In [22]:
range(0,ssm['run_information']['spin_up_days'])

range(0, 5)

In [27]:
daily_values = [1,2,3,4,5,6,7,8,9]
daily_values_clean = numpy.delete(
    daily_values,range(0,ssm['run_information']['spin_up_days']),0)
daily_values_clean

array([6, 7, 8, 9])

In [29]:
for day in range(5):
        model_day = day + ssm['run_information']['spin_up_days'] 
        model_date = dti[model_day]
        print(day,model_day, model_date)

0 5 2014-01-06 00:00:00
1 6 2014-01-07 00:00:00
2 7 2014-01-08 00:00:00
3 8 2014-01-09 00:00:00
4 9 2014-01-10 00:00:00


In [30]:
f"{model_day+1}"

'10'